# Menghitung Akurasi Tokenizer  (TK 1)

Nama: Gabriel Enrique

NPM: 1906293032

### Steps

1. Create input file for tokenizer
    - From Indonesian-PUD (ID-PUD)
    - From Indonesian-GSD5 (ID-GSD)
1. Run tokenizer on both input files
    - Bahasa
    - Aksara
1. Process the results to get score

## Imports

In [1]:
import re
import time

from conllu import parse
from humanfriendly import format_timespan

## Create Input File

Kita akan menggunakan bantuan library Conllu untuk membaca dataset `.conllu`.

In [2]:
def get_sentences(conllu_filenames, output_filename):
    sentences = []
    for conllu_filename in conllu_filenames:
        with open(conllu_filename, "r") as conllu_file:
            data = conllu_file.read()
        sentences += parse(data)
    
    with open(output_filename, "w") as output_file:
        for sentence in sentences:
            text = sentence.metadata["text"]
            output_file.write(f"{text}\n")

### Indonesian-PUD (ID-PUD)

In [3]:
get_sentences(["pud/id_pud-ud-test.conllu"], "pud_dataset.txt")

### Indonesian-GSD (ID-GSD)

In [4]:
get_sentences([
    "gsd/id_gsd-ud-dev.conllu",
    "gsd/id_gsd-ud-test.conllu",
    "gsd/id_gsd-ud-train.conllu",
], "gsd_dataset.txt")

## Run Tokenizer

### Bahasa

Kita akan menjalankan *shell command* dari Jupyter Notebook ini. Oleh karena itu, kita harus melakukan *character escaping* terlebih dahulu sebelum memberikan string sebagai input pada command.

In [15]:
# load pud dataset
with open("pud_dataset.txt", "r") as pud_dataset_file:
    pud_dataset_bahasa = pud_dataset_file.readlines()
    pud_dataset_bahasa = list(map(lambda x : x.strip("\n"), pud_dataset_bahasa)) # remove trailing \n
    pud_dataset_bahasa = list(map(lambda x : re.sub("'", "\\'", x), pud_dataset_bahasa)) # escape '
    pud_dataset_bahasa = list(map(lambda x : re.sub('"', '\\"', x), pud_dataset_bahasa)) # escape "
    pud_dataset_bahasa = list(map(lambda x : re.sub('`', '\`', x), pud_dataset_bahasa)) # escape `
    print(f"Loaded {len(pud_dataset_bahasa)} sentence(s) from 'pud_dataset.txt'")

# load gsd dataset
with open("gsd_dataset.txt", "r") as gsd_dataset_file:
    gsd_dataset_bahasa = gsd_dataset_file.readlines()
    gsd_dataset_bahasa = list(map(lambda x : x.strip("\n"), gsd_dataset_bahasa)) # remove trailing \n
    gsd_dataset_bahasa = list(map(lambda x : re.sub("'", "\\'", x), gsd_dataset_bahasa)) # escape '
    gsd_dataset_bahasa = list(map(lambda x : re.sub('"', '\\"', x), gsd_dataset_bahasa)) # escape "
    gsd_dataset_bahasa = list(map(lambda x : re.sub('`', '\`', x), gsd_dataset_bahasa)) # escape "
    print(f"Loaded {len(gsd_dataset_bahasa)} sentence(s) from 'gsd_dataset.txt'")

Loaded 1000 sentence(s) from 'pud_dataset.txt'
Loaded 5598 sentence(s) from 'gsd_dataset.txt'


Karena kita hanya ingin mendapatkan hasil tokenisasi-nya saja (tanpa POS tagging) kita tidak harus menggunakan `tag.sh`. Kita hanya cukup menggunakan `Tokenizer.pl`.

Kita akan sedikit memodifikasi `Tokenizer.pl` pada bagian input dan output filepath. Hasil modifikasi `Tokenizer.pl` ini akan disimpan di dalam file `bahasa_tokenizer.pl`.

In [16]:
# tokenize pud dataset
bahasa_pud_sentences = []

input_filename = "bahasa_tokenizer_input.txt"
output_filename = "bahasa_tokenizer_output.txt"

s = time.time()
for text in pud_dataset_bahasa:
    ! echo "$text" > "$input_filename"
    ! perl bahasa_tokenizer.pl
    with open(output_filename, "r") as output_file:
        sentence_tokens = output_file.readlines()
        sentence_tokens = list(map(lambda x : x.strip("\n"), sentence_tokens)) # remove trailing \n
        sentence_tokens = list(map(lambda x : x.strip(), sentence_tokens))
        sentence_tokens = list(filter(lambda x : len(x) > 0, sentence_tokens)) # filter out empty lines
        bahasa_pud_sentences.append(sentence_tokens)
print(f"Completed in {format_timespan(int(time.time() - s))}")

Completed in 4 minutes and 39 seconds


In [18]:
# tokenize gsd dataset
bahasa_gsd_sentences = []

input_filename = "bahasa_tokenizer_input.txt"
output_filename = "bahasa_tokenizer_output.txt"

s = time.time()
for text in gsd_dataset_bahasa:
    ! echo "$text" > "$input_filename"
    ! perl bahasa_tokenizer.pl
    with open(output_filename, "r") as output_file:
        sentence_tokens = output_file.readlines()
        sentence_tokens = list(map(lambda x : x.strip("\n"), sentence_tokens)) # remove trailing \n
        sentence_tokens = list(map(lambda x : x.strip(), sentence_tokens))
        sentence_tokens = list(filter(lambda x : len(x) > 0, sentence_tokens)) # filter out empty lines
        bahasa_gsd_sentences.append(sentence_tokens)
print(f"Completed in {format_timespan(int(time.time() - s))}")

Completed in 26 minutes and 42 seconds


### Aksara

Berbeda dari sebelumnya, di sini kita tidak perlu melakukan *character escaping* karena kita akan bekerja dengan kernel Python-nya langsung.

In [5]:
# load pud dataset
with open("pud_dataset.txt", "r") as pud_dataset_file:
    pud_dataset_aksara = pud_dataset_file.readlines()
    pud_dataset_aksara = list(map(lambda x : x.strip("\n"), pud_dataset_aksara)) # remove trailing \n
    print(f"Loaded {len(pud_dataset_aksara)} sentence(s) from 'pud_dataset.txt'")

# load gsd dataset
with open("gsd_dataset.txt", "r") as gsd_dataset_file:
    gsd_dataset_aksara = gsd_dataset_file.readlines()
    gsd_dataset_aksara = list(map(lambda x : x.strip("\n"), gsd_dataset_aksara)) # remove trailing \n
    print(f"Loaded {len(gsd_dataset_aksara)} sentence(s) from 'gsd_dataset.txt'")

Loaded 1000 sentence(s) from 'pud_dataset.txt'
Loaded 5598 sentence(s) from 'gsd_dataset.txt'


Karena kita hanya ingin mendapatkan hasil tokenisasi-nya saja (tanpa POS tagging) kita tidak harus menggunakan perintah `python3 aksara.py`. Kita hanya cukup menggunakan class `BaseTokenizer` yang ada di dalam file `aksara/tokenizer.py`.

Karena kita tidak bisa meng-import `BaseTokenizer` secara langsung, kita akan memindahkan class `BaseTokenizer` ke dalam file `aksara_tokenizer.py`.

In [6]:
from aksara_tokenizer import BaseTokenizer

aksara_tokenizer = BaseTokenizer()

In [7]:
# tokenize pud dataset
aksara_pud_sentences = []

s = time.time()
for text in pud_dataset_aksara:
    sentence_tokens, spaceafterflags = aksara_tokenizer.tokenize(text)
    sentence_tokens = list(map(lambda x : x.strip(), sentence_tokens))
    aksara_pud_sentences.append(sentence_tokens)
print(f"Completed in {format_timespan(int(time.time() - s))}")

Completed in 0 seconds


In [8]:
# tokenize gsd dataset
aksara_gsd_sentences = []

s = time.time()
for text in gsd_dataset_aksara:
    sentence_tokens, spaceafterflags = aksara_tokenizer.tokenize(text)
    sentence_tokens = list(map(lambda x : x.strip(), sentence_tokens))
    aksara_gsd_sentences.append(sentence_tokens)
print(f"Completed in {format_timespan(int(time.time() - s))}")

Completed in 0 seconds


## Tokenizer Accuracy

Kita akan mengambil *gold standard* dari kedua dataset terlebih dahulu.

In [9]:
def load_gold_standard(gold_standard_filenames):
    sentences = []
    for gold_standard_filename in gold_standard_filenames:
        with open(gold_standard_filename, "r") as gold_standard_file:
            data = gold_standard_file.read()
        sentences += parse(data)
    
    sentences_tokens = []
    for sentence in sentences:
        token_list = list(filter(lambda x : type(x["id"]) == int, sentence)) # filter out multi-word tokens
        token_list = list(map(lambda x : x["form"], token_list))
        sentences_tokens.append(token_list)
    
    return sentences_tokens

In [10]:
# load pud gold standard
pud_gold_standard = load_gold_standard(["pud/id_pud-ud-test.conllu"])
print(f"Loaded {len(pud_gold_standard)} GS sentence(s) from PUD dataset")

# load gsd gold standard
gsd_gold_standard = load_gold_standard([
    "gsd/id_gsd-ud-dev.conllu",
    "gsd/id_gsd-ud-test.conllu",
    "gsd/id_gsd-ud-train.conllu",
])
print(f"Loaded {len(gsd_gold_standard)} GS sentence(s) from GSD dataset")

Loaded 1000 GS sentence(s) from PUD dataset
Loaded 5598 GS sentence(s) from GSD dataset


File `akurasi_tokenizer.py` berisi modul-modul yang dapat digunakan untuk menghitung akurasi tokenizer berdasarkan hasil tokenisasinya.

In [11]:
from akurasi_tokenizer import get_scores

### Bahasa

In [17]:
# bahasa on pud
total_correct, total_tokens, wrong_indexes = get_scores(pud_gold_standard, bahasa_pud_sentences)

print("Accuracy (Bahasa PUD):", total_correct / total_tokens)
print("Wrongs:", *wrong_indexes)

Accuracy (Bahasa PUD): 0.9090301347320786
Wrongs: 0 2 3 6 7 8 9 10 11 12 13 17 19 20 21 22 23 24 25 26 27 29 30 34 35 36 39 40 43 44 45 49 50 55 56 58 59 61 65 67 68 70 71 72 73 74 76 81 82 83 84 85 86 87 88 90 91 93 94 95 97 98 99 100 101 103 104 105 106 107 109 112 114 115 116 120 122 126 127 128 129 134 138 139 142 143 144 146 147 151 153 156 157 158 160 161 162 163 164 166 167 168 170 175 177 180 186 189 192 193 195 197 198 200 201 202 203 208 211 213 214 215 217 221 222 223 226 227 231 232 233 237 244 245 248 250 251 252 254 255 256 258 259 261 262 265 266 267 268 269 271 272 273 274 279 282 283 285 286 290 292 296 298 301 302 308 310 313 318 321 324 325 326 328 329 330 331 333 336 337 338 339 340 341 347 348 349 350 351 353 354 355 358 359 360 361 364 365 368 371 373 374 379 385 388 389 390 392 396 398 399 400 401 402 404 408 410 414 415 418 419 421 422 426 427 430 431 433 434 435 436 439 440 441 448 450 454 459 460 461 466 467 468 469 470 471 472 473 478 481 484 485 487 488 492 

In [19]:
# bahasa on gsd
total_correct, total_tokens, wrong_indexes = get_scores(gsd_gold_standard, bahasa_gsd_sentences)

print("Accuracy (Bahasa GSD):", total_correct / total_tokens)
print("Wrongs:", *wrong_indexes)

Accuracy (Bahasa GSD): 0.8881667909622114
Wrongs: 2 3 4 5 7 9 11 13 15 17 18 19 21 22 23 24 34 36 37 39 41 43 45 46 47 49 50 52 55 56 59 60 61 62 63 65 68 69 70 71 76 77 78 81 82 84 87 89 91 92 96 99 101 102 104 105 106 107 109 110 111 113 114 117 118 124 125 126 127 128 130 131 132 134 135 140 142 143 144 147 148 149 150 151 152 153 155 158 160 161 162 163 164 166 168 170 171 172 173 175 176 177 180 184 186 187 188 190 191 192 200 202 203 204 205 206 211 212 216 217 219 220 221 223 224 226 227 228 231 233 235 237 238 239 241 242 243 244 246 249 251 252 256 257 258 259 263 264 265 266 267 268 271 274 275 276 284 285 287 291 298 299 300 301 302 303 308 309 311 312 313 315 317 319 321 323 324 326 328 329 334 335 338 339 341 342 345 347 348 349 350 351 358 361 363 368 370 372 373 374 375 381 382 384 386 387 389 390 391 395 396 398 400 403 405 406 407 408 409 410 412 414 417 419 421 423 424 427 428 430 431 432 436 437 438 439 440 441 442 443 444 446 448 449 450 451 454 455 456 457 459 460 

### Aksara

In [12]:
# aksara on pud
total_correct, total_tokens, wrong_indexes = get_scores(pud_gold_standard, aksara_pud_sentences)

print("Accuracy (Aksara PUD):", total_correct / total_tokens)
print("Wrongs:", *wrong_indexes)

Accuracy (Aksara PUD): 0.9562892111488224
Wrongs: 0 2 3 7 8 9 10 12 20 21 22 24 29 30 34 35 40 43 45 56 58 59 65 68 70 71 73 83 85 86 87 88 93 94 95 97 98 101 104 105 106 114 122 126 129 139 143 144 147 156 157 158 160 161 162 164 166 167 168 170 177 180 189 192 193 195 197 198 202 208 211 213 214 215 217 222 223 226 231 232 233 244 248 250 251 252 254 255 256 259 265 266 267 268 271 272 273 274 283 285 286 290 296 298 301 302 308 310 313 318 321 325 326 330 333 337 339 347 348 349 353 354 355 358 359 361 364 365 371 373 374 379 385 392 398 401 402 404 408 418 419 421 427 430 431 434 435 436 439 440 441 448 450 459 460 467 468 469 470 471 472 478 481 487 488 492 493 501 508 512 514 522 526 528 529 532 538 539 542 547 548 549 551 554 555 561 562 567 572 576 578 580 581 586 589 591 594 599 606 611 613 615 629 631 632 637 641 643 648 650 656 657 658 662 666 668 671 673 674 675 676 677 678 680 682 683 684 686 687 688 690 691 696 702 703 704 711 712 718 719 728 729 733 735 736 737 738 740 7

In [13]:
# aksara on gsd
total_correct, total_tokens, wrong_indexes = get_scores(gsd_gold_standard, aksara_gsd_sentences)

print("Accuracy (Aksara GSD):", total_correct / total_tokens)
print("Wrongs:", *wrong_indexes)

Accuracy (Aksara GSD): 0.960129813720589
Wrongs: 3 11 13 15 18 19 21 22 23 24 34 36 37 39 45 46 47 49 50 55 62 63 65 71 78 81 82 84 87 89 91 99 101 102 104 109 110 118 124 125 126 127 128 134 135 142 143 147 148 150 152 153 155 158 160 162 166 168 171 173 175 177 190 191 200 202 203 204 206 212 217 219 221 223 224 226 227 228 233 237 238 241 242 243 244 246 249 251 252 253 258 259 263 265 266 267 276 282 284 285 287 298 299 302 303 311 315 317 321 323 324 335 338 341 342 345 347 348 349 350 351 358 361 368 372 373 384 386 388 389 390 391 403 405 406 407 408 409 410 412 417 419 421 423 424 428 439 440 441 442 443 444 446 450 451 455 456 457 460 462 463 464 467 469 475 479 480 481 484 488 491 495 496 503 504 509 512 514 516 519 520 521 522 528 530 532 535 538 539 542 545 546 551 553 561 565 568 572 577 581 584 585 586 588 594 598 600 604 605 607 609 611 614 616 618 620 628 634 635 636 642 645 646 647 648 650 655 656 660 666 667 672 675 676 678 682 686 690 691 695 703 705 706 711 714 719 

### Error Analysis

In [34]:
# bahasa errors
print("=== Bahasa on PUD ===")
print(pud_dataset_bahasa[19], pud_gold_standard[19], bahasa_pud_sentences[19], sep="\n")
print()

print(pud_dataset_bahasa[44], pud_gold_standard[44], bahasa_pud_sentences[44], sep="\n")
print()


print("=== Bahasa on GSD ===")
print(gsd_dataset_bahasa[2], gsd_gold_standard[2], bahasa_gsd_sentences[2], sep="\n")
print()

print(gsd_dataset_bahasa[19], gsd_gold_standard[19], bahasa_gsd_sentences[19], sep="\n")
print()

=== Bahasa on PUD ===
“Kami telah meminta bangsa-bangsa lain untuk membantu kami mengisi kebun binatang dengan berbagai spesies binatang, termasuk babi,” ujar Saqib.
['“', 'Kami', 'telah', 'meminta', 'bangsa-bangsa', 'lain', 'untuk', 'membantu', 'kami', 'mengisi', 'kebun', 'binatang', 'dengan', 'berbagai', 'spesies', 'binatang', ',', 'termasuk', 'babi', ',', '”', 'ujar', 'Saqib', '.']
['“Kami', 'telah', 'meminta', 'bangsa-bangsa', 'lain', 'untuk', 'membantu', 'kami', 'mengisi', 'kebun', 'binatang', 'dengan', 'berbagai', 'spesies', 'binatang', ',', 'termasuk', 'babi', ',”', 'ujar', 'Saqib', '.']

\"Sayangnya, lagi-lagi karena nila setitik rusak susu sebelanga,\" tulis Jesse LaBrocca, pendiri Hack Forums, dalam sebuah pesan yang menjelaskan mengapa bagian itu ditutup.
['"', 'Sayangnya', ',', 'lagi-lagi', 'karena', 'nila', 'setitik', 'rusak', 'susu', 'sebelanga', ',', '"', 'tulis', 'Jesse', 'LaBrocca', ',', 'pendiri', 'Hack', 'Forums', ',', 'dalam', 'sebuah', 'pesan', 'yang', 'menjelaskan

In [14]:
# aksara errors
print("=== Aksara on PUD ===")
print(pud_dataset_aksara[744], pud_gold_standard[744], aksara_pud_sentences[744], sep="\n")
print()

print(pud_dataset_aksara[70], pud_gold_standard[70], aksara_pud_sentences[70], sep="\n")
print()


print("=== Aksara on GSD ===")
print(gsd_dataset_aksara[528], gsd_gold_standard[528], aksara_gsd_sentences[528], sep="\n")
print()

print(gsd_dataset_aksara[19], gsd_gold_standard[19], aksara_gsd_sentences[19], sep="\n")
print()

=== Aksara on PUD ===
Wright adalah teman sesama direktur Garth Jennings, dan tampil cameo dalam filmnya The Hitchhiker's Guide to the Galaxy dan Son of Rambow.
['Wright', 'adalah', 'teman', 'sesama', 'direktur', 'Garth', 'Jennings', ',', 'dan', 'tampil', 'cameo', 'dalam', 'film', 'nya', 'The', 'Hitchhiker', "'s", 'Guide', 'to', 'the', 'Galaxy', 'dan', 'Son', 'of', 'Rambow', '.']
['Wright', 'adalah', 'teman', 'sesama', 'direktur', 'Garth', 'Jennings', ',', 'dan', 'tampil', 'cameo', 'dalam', 'filmnya', 'The', 'Hitchhiker', "'s", 'Guide', 'to', 'the', 'Galaxy', 'dan', 'Son', 'of', 'Rambow', '.']

Seagal, yang neneknya berasal dari Vladivostok di Timur Jauh Rusia, sering melakukan perjalanan ke Rusia dalam beberapa tahun terakhir dan mengunjungi Kamchatka dan Sakhalin di bulan September.
['Seagal', ',', 'yang', 'nenek', 'nya', 'berasal', 'dari', 'Vladivostok', 'di', 'Timur', 'Jauh', 'Rusia', ',', 'sering', 'melakukan', 'perjalanan', 'ke', 'Rusia', 'dalam', 'beberapa', 'tahun', 'terakhir',